In [3]:
'''
import all the required packages...
relativedata used for adding/removing month from current date
tensorflow_probability for tfp model 
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.api as sm
import itertools
import warnings
warnings.filterwarnings("ignore")
from dateutil import relativedelta
import tensorflow as tf
import tensorflow_probability as tfp

In [9]:
df = pd.read_excel('brand_level_master_reduced_holiday.xlsx')

In [10]:
X=df[['Date','Key','Volume','avg_T','precipitation','GTO_Vol','Disc_Vol']]
X = pd.concat([X,df[['Industrial_Production_Index_month','Unemployment_Rate_quarter','Terms_of_Trade_annual',
                    'Consumer_Price_Index_month','Nominal_GDP_Per_Capita_US_quarter']]],axis=1)

In [11]:
#holiday variables....
X = pd.concat([X,df.iloc[:,697:]],axis=1)

In [13]:
#get all the unique unique brand as we are running brand wise....
X.Date = pd.to_datetime(X.Date,format='%d-%m-%Y')
Brand_list = ['BUD']
#Brand_list = X.Key.unique()

In [14]:
# Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`.
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential([
          tfp.layers.VariableLayer(2 * n, dtype=dtype),
          tfp.layers.DistributionLambda(lambda t: tfd.Independent(
              tfd.Normal(loc=t[..., :n],
                         scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
              reinterpreted_batch_ndims=1)),
    ])

In [15]:
# Specify the prior over `keras.layers.Dense` `kernel` and `bias`.
def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
      tfp.layers.VariableLayer(n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t, scale=1),
          reinterpreted_batch_ndims=1)),
      ])

In [46]:
final = pd.DataFrame()
#getting distribution...
tfd = tfp.distributions
negloglik = lambda y, rv_y: -rv_y.log_prob(y)
for brand_name in Brand_list:
    print(brand_name)
    brand_df = X.loc[X.Key == brand_name]
    brand_df.set_index('Date',inplace=True)
    tmp = []
    forecast = pd.DataFrame()
    Actuals = pd.DataFrame()

    for i in range(1,0,-1):
        train_date = brand_df.index.max() - relativedelta.relativedelta(months=i)
        test_date = train_date + relativedelta.relativedelta(months=1)
        x = brand_df.drop(columns=['Volume','Key'])
        y = brand_df[['Volume']]
        train_x = x[:train_date]
        train_y = y[:train_date][['Volume']]
        test_x = x[test_date:]
        test_y = y[test_date:][['Volume']]

#         model = tf.keras.Sequential([tf.keras.layers.Dense(1),
#                                      tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t, scale=1)),
#                                     ])
    
        model = tf.keras.Sequential([tfp.layers.DenseVariational(1 + 1, posterior_mean_field, prior_trainable, kl_weight=1/x.shape[0]),
                                     tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., :1],scale=1e-3)),
                                    ])
        
#         model = tf.keras.Sequential([tfp.layers.DenseVariational(1+1, posterior_mean_field, prior_trainable, kl_weight=1/train_x.shape[0]),
#                                      tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t, scale=1)),
#                                     ])

        # Do inference.
        model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01), loss=negloglik)
        model.fit(np.array(train_x), np.array(train_y), epochs=1000, verbose=True);

        # Profit.
        yhat = model(np.array(test_x))
        print([print(np.squeeze(w.numpy())) for w in model.weights])
        print(yhat.mean())
        
#         forecast[str(brand_name)+str('le-')+str(test_date.month)] = np.array(yhat.mean()[0])
#         Actuals[str(brand_name)+str('Ac-')+str(test_date.month)] =  np.array(test_y)[0]
        
#     tmp1 = forecast.melt()
#     tmp2 = Actuals.melt()
#     tmp1.columns = [str(brand_name+"le"),str(brand_name+'forecast')]
#     tmp2.columns = [str(brand_name+"Ac"),str(brand_name+'Actuals')]
#     final = pd.concat([final,tmp1,tmp2],axis=1)

BUD
                avg_T  precipitation      GTO_Vol    Disc_Vol  \
Date                                                            
2015-01-01  39.465496       1.022196  1076.192976  281.026861   
2015-02-01  42.960097       1.305528  1090.856700  355.031815   
2015-03-01  51.857607       1.861202  1059.875005  305.554055   
2015-04-01  61.168728       2.966924  1063.912648  313.124183   
2015-05-01  70.335217       5.915735  1016.317537  286.617582   
2015-06-01  76.431403       6.872861  1016.960603  268.540622   
2015-07-01  79.176431       6.349210   999.523008  280.504000   
2015-08-01  78.957539       6.197718  1019.681751  282.530335   
2015-09-01  72.107727       4.311215  1048.486466  302.705975   
2015-10-01  63.259857       2.838055  1051.152598  318.929638   
2015-11-01  50.610878       3.391475  1098.111797  344.526152   
2015-12-01  40.894636       2.133515  1108.523302  299.101415   
2016-01-01  35.092536       2.745716  1106.798985  292.902103   
2016-02-01  41.053923

Train on 57 samples
Epoch 1/1000
57/57 [==============================] - 2s 38ms/sample - loss: 1425001730737277696.0000
Epoch 2/1000
57/57 [==============================] - 0s 175us/sample - loss: 1440298134091687680.0000
Epoch 3/1000
57/57 [==============================] - 0s 192us/sample - loss: 1410909393886090240.0000
Epoch 4/1000
57/57 [==============================] - 0s 175us/sample - loss: 1428884865826013696.0000
Epoch 5/1000
57/57 [==============================] - 0s 385us/sample - loss: 1428397145615545344.0000
Epoch 6/1000
57/57 [==============================] - 0s 175us/sample - loss: 1416313606863465728.0000
Epoch 7/1000
57/57 [==============================] - 0s 175us/sample - loss: 1412916569241063424.0000
Epoch 8/1000
57/57 [==============================] - 0s 192us/sample - loss: 1422979369583550976.0000
Epoch 9/1000
57/57 [==============================] - 0s 420us/sample - loss: 1416729292183846400.0000
Epoch 10/1000
57/57 [==============================] -

57/57 [==============================] - 0s 175us/sample - loss: 1411254570612130304.0000
Epoch 80/1000
57/57 [==============================] - 0s 192us/sample - loss: 1408236080858155008.0000
Epoch 81/1000
57/57 [==============================] - 0s 420us/sample - loss: 1405399446951720192.0000
Epoch 82/1000
57/57 [==============================] - 0s 157us/sample - loss: 1391620331064285696.0000
Epoch 83/1000
57/57 [==============================] - 0s 157us/sample - loss: 1384155922641623296.0000
Epoch 84/1000
57/57 [==============================] - 0s 210us/sample - loss: 1405995382253974784.0000
Epoch 85/1000
57/57 [==============================] - 0s 210us/sample - loss: 1388286582874352384.0000
Epoch 86/1000
57/57 [==============================] - 0s 261us/sample - loss: 1377393938186849280.0000
Epoch 87/1000
57/57 [==============================] - 0s 194us/sample - loss: 1384943122331706880.0000
Epoch 88/1000
57/57 [==============================] - 0s 247us/sample - loss:

57/57 [==============================] - 0s 157us/sample - loss: 1355826368992611072.0000
Epoch 158/1000
57/57 [==============================] - 0s 140us/sample - loss: 1388558017573830144.0000
Epoch 159/1000
57/57 [==============================] - 0s 175us/sample - loss: 1374741187955321344.0000
Epoch 160/1000
57/57 [==============================] - 0s 175us/sample - loss: 1374333871943843840.0000
Epoch 161/1000
57/57 [==============================] - 0s 175us/sample - loss: 1379659609689996544.0000
Epoch 162/1000
57/57 [==============================] - 0s 157us/sample - loss: 1365172077978543872.0000
Epoch 163/1000
57/57 [==============================] - 0s 175us/sample - loss: 1378464081280357120.0000
Epoch 164/1000
57/57 [==============================] - 0s 140us/sample - loss: 1357741105800930560.0000
Epoch 165/1000
57/57 [==============================] - 0s 157us/sample - loss: 1380271553013516032.0000
Epoch 166/1000
57/57 [==============================] - 0s 140us/sampl

57/57 [==============================] - 0s 140us/sample - loss: 1330077436647861248.0000
Epoch 236/1000
57/57 [==============================] - 0s 157us/sample - loss: 1353749972629282304.0000
Epoch 237/1000
57/57 [==============================] - 0s 140us/sample - loss: 1338893516187355648.0000
Epoch 238/1000
57/57 [==============================] - 0s 157us/sample - loss: 1341446608710358272.0000
Epoch 239/1000
57/57 [==============================] - 0s 157us/sample - loss: 1350709885669934336.0000
Epoch 240/1000
57/57 [==============================] - 0s 157us/sample - loss: 1351374646542151936.0000
Epoch 241/1000
57/57 [==============================] - 0s 140us/sample - loss: 1325131881570319104.0000
Epoch 242/1000
57/57 [==============================] - 0s 157us/sample - loss: 1342221441305839360.0000
Epoch 243/1000
57/57 [==============================] - 0s 140us/sample - loss: 1341436185050781952.0000
Epoch 244/1000
57/57 [==============================] - 0s 140us/sampl

...

57/57 [==============================] - 0s 170us/sample - loss: 1196386660445916416.0000
Epoch 704/1000
57/57 [==============================] - 0s 210us/sample - loss: 1197802621647247104.0000
Epoch 705/1000
57/57 [==============================] - 0s 245us/sample - loss: 1205091208274668544.0000
Epoch 706/1000
57/57 [==============================] - 0s 235us/sample - loss: 1207951396083155712.0000
Epoch 707/1000
57/57 [==============================] - 0s 262us/sample - loss: 1207699046108532736.0000
Epoch 708/1000
57/57 [==============================] - 0s 182us/sample - loss: 1194556853677213696.0000
Epoch 709/1000
57/57 [==============================] - 0s 228us/sample - loss: 1193296675912828928.0000
Epoch 710/1000
57/57 [==============================] - 0s 175us/sample - loss: 1181286211282221824.0000
Epoch 711/1000
57/57 [==============================] - 0s 216us/sample - loss: 1183517224056996608.0000
Epoch 712/1000
57/57 [==============================] - 0s 188us/sampl

57/57 [==============================] - 0s 227us/sample - loss: 1175216914330527232.0000
Epoch 782/1000
57/57 [==============================] - 0s 195us/sample - loss: 1176961865807114496.0000
Epoch 783/1000
57/57 [==============================] - 0s 175us/sample - loss: 1159836294654575104.0000
Epoch 784/1000
57/57 [==============================] - 0s 192us/sample - loss: 1179840302856292352.0000
Epoch 785/1000
57/57 [==============================] - 0s 175us/sample - loss: 1182842676084565760.0000
Epoch 786/1000
57/57 [==============================] - 0s 157us/sample - loss: 1152151711439660032.0000
Epoch 787/1000
57/57 [==============================] - 0s 157us/sample - loss: 1176228694093003520.0000
Epoch 788/1000
57/57 [==============================] - 0s 175us/sample - loss: 1165392452423037952.0000
Epoch 789/1000
57/57 [==============================] - 0s 157us/sample - loss: 1176736550315760384.0000
Epoch 790/1000
57/57 [==============================] - 0s 175us/sampl

57/57 [==============================] - 0s 175us/sample - loss: 1157526249659133952.0000
Epoch 860/1000
57/57 [==============================] - 0s 157us/sample - loss: 1149978784706799744.0000
Epoch 861/1000
57/57 [==============================] - 0s 140us/sample - loss: 1134991102998823680.0000
Epoch 862/1000
57/57 [==============================] - 0s 192us/sample - loss: 1152939333091443200.0000
Epoch 863/1000
57/57 [==============================] - 0s 157us/sample - loss: 1151469671838660096.0000
Epoch 864/1000
57/57 [==============================] - 0s 157us/sample - loss: 1132778350315182720.0000
Epoch 865/1000
57/57 [==============================] - 0s 158us/sample - loss: 1162908711113715200.0000
Epoch 866/1000
57/57 [==============================] - 0s 175us/sample - loss: 1165182370954631680.0000
Epoch 867/1000
57/57 [==============================] - 0s 192us/sample - loss: 1144527359037696896.0000
Epoch 868/1000
57/57 [==============================] - 0s 175us/sampl

57/57 [==============================] - 0s 227us/sample - loss: 1129432157871936000.0000
Epoch 938/1000
57/57 [==============================] - 0s 106us/sample - loss: 1134298524000181248.0000
Epoch 939/1000
57/57 [==============================] - 0s 175us/sample - loss: 1130015026828771968.0000
Epoch 940/1000
57/57 [==============================] - 0s 227us/sample - loss: 1137116227499182592.0000
Epoch 941/1000
57/57 [==============================] - 0s 210us/sample - loss: 1126309585839617280.0000
Epoch 942/1000
57/57 [==============================] - 0s 210us/sample - loss: 1138057583059658112.0000
Epoch 943/1000
57/57 [==============================] - 0s 192us/sample - loss: 1133383797839743488.0000
Epoch 944/1000
57/57 [==============================] - 0s 175us/sample - loss: 1122194234377337344.0000
Epoch 945/1000
57/57 [==============================] - 0s 245us/sample - loss: 1145348566429531008.0000
Epoch 946/1000
57/57 [==============================] - 0s 788us/sampl

[ 1.7705336e+01  9.8301813e-02  1.7684441e+01 -2.0816758e-02
  1.7231766e+01 -5.4951742e-02  1.7783884e+01 -4.5757569e-02
  1.7551775e+01  7.8698069e-02  1.7584908e+01 -8.7594464e-02
  1.7342215e+01  4.7708094e-02  1.7704773e+01 -1.0289725e-02
  1.7660852e+01  9.4363363e-03  1.6196070e+01  8.9246929e-02
  1.6465469e+01  5.1015574e-02  1.6696398e+01  8.0454331e-05
  1.6651222e+01 -1.2112681e-02  1.5903638e+01  5.6654532e-02
  1.6364275e+01  5.1597148e-02  1.4575155e+01 -2.2036290e-02
  1.4360532e+01  1.9386635e-03  1.6418352e+01 -8.9441165e-03
  1.6201252e+01  6.4410262e-02  1.6276466e+01 -1.9227982e-02
  1.7408546e+01 -8.5477373e-03  1.6921232e+01  6.4492248e-02
  1.7483028e+01  1.0286136e-02]
[None, None]
tf.Tensor([[240945.6]], shape=(1, 1), dtype=float32)


In [47]:
# test_x
yhat = model.predict(np.array(test_x))
print(yhat)
print(yhat.mean())
print(test_y)

[[233331.83]]
233331.83
                 Volume
Date                   
2019-10-01  1228506.813


In [7]:
final_numeric= final.ix[:,final.applymap(np.isreal).all(axis=0)]
final_numeric.to_csv('TFP_probability_layer_all.csv')